# 🌍 地震學第一天：地震資料分析入門

歡迎！這個筆記本將引導你完成第一次地震學分析。

## 🎯 學習目標
- 下載真實地震資料
- 視覺化呈現地震波形
- 處理和過濾資料
- 使用 AI 輔助學習

## 💡 如何使用本筆記本
1. 按 `Shift + Enter` 執行每個儲存格
2. 閱讀程式碼儲存格之間的說明
3. 嘗試修改程式碼進行探索
4. 如果遇到困難，請詢問 Gemini AI！

---

## 步驟 1：安裝所需函式庫

如果你使用 Google Colab，請先執行此儲存格：

In [ ]:
# Only run this in Google Colab
!pip install obspy -q
print("✅ Installation complete!")

## 步驟 2：匯入函式庫

讓我們匯入所需的工具：

In [ ]:
# Core seismology library
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

# Plotting and numerical tools
import matplotlib.pyplot as plt
import numpy as np

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ All libraries imported successfully!")
print("🎉 You're ready to analyze earthquakes!")

## 步驟 3：連接到 IRIS 資料中心

IRIS 提供來自世界各地的地震資料：

In [ ]:
# Create client connection
client = Client("IRIS")
print("📡 Connected to IRIS data center!")
print("🌐 You can now download data from thousands of stations worldwide")

## 步驟 4：下載真實地震資料

讓我們取得一個重大地震的資料。試試這個範例，或請 AI 幫你找一個最近的地震！

In [ ]:
# Define earthquake time and station
# Example: 2024 Turkey Earthquake
earthquake_time = UTCDateTime("2024-02-06T01:17:00")
duration = 600  # seconds (10 minutes)

print(f"⏳ Downloading data for earthquake at {earthquake_time}...")

# Download waveform data
stream = client.get_waveforms(
    network="IU",           # Global network
    station="ANTO",         # Ankara, Turkey
    location="00",          # Location code
    channel="BHZ",          # Broadband vertical
    starttime=earthquake_time,
    endtime=earthquake_time + duration
)

print(f"✅ Success! Downloaded {len(stream)} trace(s)")
print("\n📊 Data Information:")
print(stream)

## 步驟 5：繪製原始資料

讓我們看看地震的樣子：

In [ ]:
# Quick plot using ObsPy's built-in plotting
stream.plot(figsize=(12, 4))

### �� 你看到了什麼？

- **地震前：** 小的背景雜訊（微震）
- **P 波到達：** 第一個尖銳的峰值（壓縮波）
- **S 波到達：** 較大振幅的波（剪切波）
- **表面波：** 長週期振盪

💡 **試著詢問 Gemini：** "P 波、S 波和表面波之間有什麼差異？"

## 步驟 6：取得詳細統計資訊

In [ ]:
# Extract the trace (single recording)
trace = stream[0]

print("📊 Recording Details:")
print(f"  Station: {trace.stats.station}")
print(f"  Channel: {trace.stats.channel}")
print(f"  Sampling rate: {trace.stats.sampling_rate} Hz")
print(f"  Number of samples: {trace.stats.npts}")
print(f"  Start time: {trace.stats.starttime}")
print(f"  End time: {trace.stats.endtime}")
print(f"  Duration: {trace.stats.endtime - trace.stats.starttime:.1f} seconds")

## 步驟 7：處理資料

讓我們透過濾波來清理資料：

In [ ]:
# Make a copy to preserve original data
stream_filtered = stream.copy()

# Apply bandpass filter (0.1 to 10 Hz)
print("🔧 Applying bandpass filter (0.1 - 10 Hz)...")
stream_filtered.filter("bandpass", freqmin=0.1, freqmax=10.0)

print("✅ Filtering complete!")
print("This removes high-frequency noise and long-period drift.")

## 步驟 8：比較原始資料與濾波後資料

In [ ]:
# Create comparison plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Plot raw data
times = stream[0].times()
ax1.plot(times, stream[0].data, 'b-', linewidth=0.5, alpha=0.8)
ax1.set_title('Raw Seismic Data', fontsize=14, fontweight='bold')
ax1.set_ylabel('Amplitude (counts)', fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot filtered data
ax2.plot(times, stream_filtered[0].data, 'r-', linewidth=0.5, alpha=0.8)
ax2.set_title('Filtered Seismic Data (0.1-10 Hz)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Time (seconds)', fontsize=11)
ax2.set_ylabel('Amplitude (counts)', fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Notice how filtering makes the earthquake signals clearer!")

## 步驟 9：計算基本參數

In [ ]:
# Get the data array
data = stream_filtered[0].data

# Calculate statistics
max_amp = np.max(np.abs(data))
mean_val = np.mean(data)
std_val = np.std(data)

# Find time of maximum amplitude
max_index = np.argmax(np.abs(data))
max_time = times[max_index]

print("📊 Waveform Statistics:")
print(f"  Maximum amplitude: {max_amp:.2e}")
print(f"  Mean value: {mean_val:.2e}")
print(f"  Standard deviation: {std_val:.2e}")
print(f"  Peak arrival time: {max_time:.2f} seconds")

## 步驟 10：放大波形到達

In [ ]:
# Plot first 200 seconds to see wave arrivals clearly
fig, ax = plt.subplots(figsize=(14, 6))

# Filter for first 200 seconds
mask = times <= 200
ax.plot(times[mask], data[mask], 'b-', linewidth=1)

# Mark potential wave arrivals (these are estimates!)
ax.axvline(x=50, color='r', linestyle='--', linewidth=2, 
           label='~P-wave arrival', alpha=0.7)
ax.axvline(x=90, color='g', linestyle='--', linewidth=2, 
           label='~S-wave arrival', alpha=0.7)

ax.set_title('Wave Arrivals - First 200 Seconds', fontsize=14, fontweight='bold')
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🌊 P-waves (Primary) arrive first - they're faster!")
print("🌊 S-waves (Secondary) arrive later - they're slower but stronger!")

## 🎉 恭喜！

你已經完成了第一次地震學分析！你已經學會：
- ✅ 下載真實地震資料
- ✅ 視覺化呈現地震圖
- ✅ 過濾地震訊號
- ✅ 計算基本參數
- ✅ 識別波形到達

---

## 🚀 挑戰練習

嘗試這些修改來加深你的理解：

### 簡單：
1. 將測站改為 `"ANMO"` (美國新墨西哥州)
2. 下載 20 分鐘的資料而不是 10 分鐘
3. 嘗試不同的濾波頻率（例如 1-5 Hz）

### 中等：
4. 下載所有三個分量 (BHE, BHN, BHZ)
5. 將它們繪製在同一張圖上
6. 計算 P 波和 S 波之間的時間差

### 進階：
7. 建立頻譜圖以查看頻率內容
8. 從多個測站下載資料
9. 比較不同距離的波形

---

## 🤖 使用 AI 學習更多

向 Gemini 詢問這些問題：
- "如何在 ObsPy 中建立頻譜圖？"
- "S-P 時間與距離之間的關係是什麼？"
- "如何一次從多個測站下載資料？"

使用 NotebookLM：
- 上傳地震學教科書章節
- 詢問："解釋地震波傳播"
- 為這個筆記本生成學習指南

---

## 📚 額外資源

- **ObsPy Documentation:** https://docs.obspy.org/
- **IRIS Data Services:** https://www.iris.edu/
- **Recent Earthquakes:** https://earthquake.usgs.gov/
- **Setup Guide:** 請參閱儲存庫中的 SETUP_GUIDE.md
- **Full Tutorial:** 請參閱 EXAMPLE_TUTORIAL.md 以獲取更多詳細資訊

---

**祝你編碼愉快！🌍📊🐍**